# Libs

In [1]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import json
import matplotlib.patheffects as PathEffects
import matplotlib.patches as patches




In [2]:
# Configurar pandas para mostrar o valor completo 
pd.set_option('display.float_format', '{:.10f}'.format)

# Importando dados

In [3]:
# df = pd.read_csv('https://media.githubusercontent.com/media/Felisouza/tcc_mba_icmc/refs/heads/main/data/data.csv')
df = pd.read_csv(r'..\data\data.csv')
df.head()

C:\Users\felipebatista\AppData\Local\Temp\ipykernel_11444\2443517200.py:2: DtypeWarning: Columns (1,11,15,26,32,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'..\data\data.csv')


,DsStatus,UltimoMotivoEndosso,VlReceitaEsperada,DtPrimeiraParcela,NrParcelas,NmProduto,GrupoProduto,NOME_SEGURADORA_FINAL,Balcao,FlgHabitacao,...,StatusData,StatusValor,DifParcela,DifData,TIPO INADIMPLÊNCIA,NOTIFICAÇÕES,STATUS,STATUS PRÊMIO,INADIMPLÊNTE?,Esteve_Inadimplente?
0,NaN,NaN,12222.4501953125,NaN,1.0000000000,2983630b65205bf108d9d69c960f67a8361b869201ad38...,8d76d9752f0ebba3365fd8bc593b811c8aa48160114978...,0feaa5e88e73d8e7eefc6d92c9bdb298a83b424e5e1902...,dc937b59892604f5a86ac96936cd7ff09e25f18ae6b758...,Fora Habitacao,...,Comissão não recebida,Comissão não recebida,12222.4500000000,-9140.0000000000,NaN,NaN,NaN,NaN,NaN,NÃO
1,NaN,NaN,12222.4501953125,NaN,1.0000000000,2983630b65205bf108d9d69c960f67a8361b869201ad38...,8d76d9752f0ebba3365fd8bc593b811c8aa48160114978...,0feaa5e88e73d8e7eefc6d92c9bdb298a83b424e5e1902...,dc937b59892604f5a86ac96936cd7ff09e25f18ae6b758...,Fora Habitacao,...,Comissão não recebida,Comissão não recebida,12222.4500000000,-9140.0000000000,NaN,NaN,NaN,NaN,NaN,NÃO
2,Confirmed,NaN,0.0000000000,2022-07-25,1.0000000000,9cdafbed10c0d51c62c70109841e569900f978bb7524ef...,54eb3384177a6f1bce36ebf478c32787a128b43144b901...,7a86589d1624833154c067fdf8379eca1c7de984bba0c3...,1c0779e41e8d0a97be3b94feda097bb3280456b4ad5363...,Massificado,...,Comissão não recebida,Comissão não recebida,0.0000000000,-899.0000000000,NaN,NaN,NaN,NaN,NaN,NÃO
3,Confirmado,NaN,527.8207397461,2016-05-26,1.0000000000,c68cb6c1bac51b9b18fe707b35d142a54e67f06780c358...,243f77a913aad6b64b4c73b481c94fce10c1554fbe86d1...,ce7112b1bb7af451acc6c360998019acfde75d2dcb1527...,dc937b59892604f5a86ac96936cd7ff09e25f18ae6b758...,Habitacao,...,Comissão não recebida,Comissão não recebida,527.8200000000,-3150.0000000000,NaN,NaN,NaN,NaN,NaN,NÃO
4,Confirmado,NaN,65.5500030518,2016-10-01,1.0000000000,f22e67f434917d011b2a3f8463d63bdf4acf6ddd14840b...,0048e821bb69ffdcd8f340d7d83aaefadea6b47377089b...,ce7112b1bb7af451acc6c360998019acfde75d2dcb1527...,dc937b59892604f5a86ac96936cd7ff09e25f18ae6b758...,Fora Habitacao,...,Comissão não recebida,Comissão não recebida,65.5500000000,-3022.0000000000,NaN,NaN,NaN,NaN,NaN,NÃO


In [ ]:
df.info()

In [ ]:
df['NrParcelas'].dtype

In [ ]:
df.describe()

In [ ]:
df['DsStatus'].value_counts()

In [ ]:
df['STATUS'].value_counts()

In [ ]:
df['STATUS PRÊMIO'].value_counts()

# Limpeza dos dados

## Arredondando valores float

In [4]:
float_cols = df.select_dtypes(include=['float64']).columns 
float_cols
# df_sem[float_cols] = df[float_cols].round(0).astype(int)

Index(['VlReceitaEsperada', 'NrParcelas', 'PremioBrutoModDiv', 'ValorParcela',
       'VlComissao_co', 'DifParcela', 'DifData', 'NOTIFICAÇÕES'],
      dtype='object')

In [5]:
df[float_cols] = df[float_cols].fillna(0).replace([float('inf'), -float('inf')], 0)

In [6]:
df[float_cols] = df[float_cols].round(0).astype(int)

## Valor prorrogado para 'NÃO'

In [7]:
df['INADIMPLÊNTE?'].replace('PRORROGADO', 'NÃO', inplace=True)

# Tirando apólices canceladas

In [8]:
df_sem_canc = (
df[
    (df['DsStatus'] != 'Cancelado')
    & (df['DsStatus'] != 'Cancelado(a)')
    & (df['STATUS'] != 'APÓLICE CANCELADA')
    & (df['STATUS'] != 'AGUARDANDO BAIXA DE CANCELAMENTO')
    & (df['STATUS PRÊMIO'] != 'EM CANCELAMENTO')
    & (df['INADIMPLÊNTE?'] != 'CANCELADA')
    & (~df['DsStatus'].isna())
    & (~df['Data_Receber'].isna())
]
)

## Coluna Inadimplente_Atual

In [9]:
df['Inadimplente_Atual'] = df['INADIMPLÊNTE?'].apply(lambda x: 1 if x == 'SIM' else 0)


## Coluna target para numérico

In [10]:
def coluna_binaria(df, coluna):
    novo_nome_coluna = coluna + '_num'
    df[novo_nome_coluna] = df[coluna].map({'SIM':1, 'NÃO':0})
    return df

In [11]:
df_sem_canc = coluna_binaria(df_sem_canc, 'INADIMPLÊNTE?')

C:\Users\felipebatista\AppData\Local\Temp\ipykernel_11444\203985392.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[novo_nome_coluna] = df[coluna].map({'SIM':1, 'NÃO':0})


In [12]:
df_sem_canc = coluna_binaria(df_sem_canc, 'Esteve_Inadimplente?')

C:\Users\felipebatista\AppData\Local\Temp\ipykernel_11444\203985392.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[novo_nome_coluna] = df[coluna].map({'SIM':1, 'NÃO':0})


## Coluna trimestre - ano

In [13]:
df_sem_canc.dtypes

DsStatus                     object
UltimoMotivoEndosso          object
VlReceitaEsperada             int32
DtPrimeiraParcela            object
NrParcelas                    int32
NmProduto                    object
GrupoProduto                 object
NOME_SEGURADORA_FINAL        object
Balcao                       object
FlgHabitacao                 object
TipoEmissao                  object
PeriodocidadePagamento       object
DtEmissao                    object
NmTomador                    object
GrupoEconomico               object
TipoOportunidade             object
PremioBrutoModDiv             int32
Parcela                       int64
ValorParcela                  int32
DataVencimento               object
Periodicidade                 int64
PeriodicidadeMes              int64
Data_Receber                 object
VlComissao_co                 int32
NmSeguradora_co              object
CanalVenda_co                object
DtFaturamento_max_co         object
Status_PG                   

In [14]:
# Converter a coluna 'Data_Receber' para o formato datetime
df_sem_canc['Data_Receber'] = pd.to_datetime(df_sem_canc['Data_Receber'])

# Criar a coluna de trimestres com valores 1, 2, 3 ou 4
df_sem_canc['trimestre'] = df_sem_canc['Data_Receber'].dt.quarter

# Preencher valores NA com 0
df_sem_canc['trimestre'] = df_sem_canc['trimestre'].fillna(0)

# Converter a coluna de trimestres para o tipo int
df_sem_canc['trimestre'] = df_sem_canc['trimestre'].astype(int)


C:\Users\felipebatista\AppData\Local\Temp\ipykernel_11444\3291830796.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sem_canc['Data_Receber'] = pd.to_datetime(df_sem_canc['Data_Receber'])
C:\Users\felipebatista\AppData\Local\Temp\ipykernel_11444\3291830796.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sem_canc['trimestre'] = df_sem_canc['Data_Receber'].dt.quarter
C:\Users\felipebatista\AppData\Local\Temp\ipykernel_11444\3291830796.py:8: SettingWithCopyWarning: 
A value is trying to be set on

## Tratando valores nulos

In [15]:
df_sem_canc.isnull().sum().sort_values(ascending=False)

UltimoMotivoEndosso         1860292
STATUS                      1857380
INADIMPLÊNTE?_num           1857159
INADIMPLÊNTE?               1857159
STATUS PRÊMIO               1857159
TIPO INADIMPLÊNCIA          1857159
TipoOportunidade            1400361
DtFaturamento_max_co         616380
PeriodocidadePagamento       440810
DtPrimeiraParcela             13835
StatusValor                       0
CanalVenda_co                     0
Status_PG                         0
StatusData                        0
DsStatus                          0
DifParcela                        0
DifData                           0
VlComissao_co                     0
NOTIFICAÇÕES                      0
Esteve_Inadimplente?              0
Esteve_Inadimplente?_num          0
NmSeguradora_co                   0
Periodicidade                     0
Data_Receber                      0
PeriodicidadeMes                  0
VlReceitaEsperada                 0
NrParcelas                        0
NmProduto                   

In [16]:
# Listar as colunas a serem removidas
colunas_para_remover = [
    'UltimoMotivoEndosso',
    'STATUS',
    'TIPO INADIMPLÊNCIA',
    'INADIMPLÊNTE?_num',
    'INADIMPLÊNTE?',
    'STATUS PRÊMIO',
    'TipoOportunidade',
    'DtFaturamento_max_co',
    'PeriodocidadePagamento',
    'DtPrimeiraParcela'
]

# Remover as colunas do DataFrame
df_sem_canc = df_sem_canc.drop(columns=colunas_para_remover)


In [17]:
df_sem_canc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1867279 entries, 2 to 1952062
Data columns (total 31 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   DsStatus                  object        
 1   VlReceitaEsperada         int32         
 2   NrParcelas                int32         
 3   NmProduto                 object        
 4   GrupoProduto              object        
 5   NOME_SEGURADORA_FINAL     object        
 6   Balcao                    object        
 7   FlgHabitacao              object        
 8   TipoEmissao               object        
 9   DtEmissao                 object        
 10  NmTomador                 object        
 11  GrupoEconomico            object        
 12  PremioBrutoModDiv         int32         
 13  Parcela                   int64         
 14  ValorParcela              int32         
 15  DataVencimento            object        
 16  Periodicidade             int64         
 17  Periodicidade

## Retirando colunas

In [18]:
colunas_para_remover = [
    'DsStatus',
    'Balcao',
    'FlgHabitacao',
    'TipoEmissao',
    'DtEmissao',
    'DataVencimento',
    'Periodicidade',
    'NmSeguradora_co',
    'CanalVenda_co',
    'Status_PG',
    'StatusData',
    'StatusValor',
    'NOTIFICAÇÕES'
]

df_sem_canc = df_sem_canc.drop(columns=colunas_para_remover)
df_sem_canc.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1867279 entries, 2 to 1952062
Data columns (total 18 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   VlReceitaEsperada         int32         
 1   NrParcelas                int32         
 2   NmProduto                 object        
 3   GrupoProduto              object        
 4   NOME_SEGURADORA_FINAL     object        
 5   NmTomador                 object        
 6   GrupoEconomico            object        
 7   PremioBrutoModDiv         int32         
 8   Parcela                   int64         
 9   ValorParcela              int32         
 10  PeriodicidadeMes          int64         
 11  Data_Receber              datetime64[ns]
 12  VlComissao_co             int32         
 13  DifParcela                int32         
 14  DifData                   int32         
 15  Esteve_Inadimplente?      object        
 16  Esteve_Inadimplente?_num  int64         
 17  trimestre    

In [19]:
# df_sem_canc['StatusValor'].unique()

## Padronizando dados financeiros

In [20]:
# Inicializar o StandardScaler
scaler = StandardScaler()

#colunas para padronizar
colunas_padronizar = ['VlReceitaEsperada', 'PremioBrutoModDiv', 'ValorParcela', 'VlComissao_co']


#Padronizando colunas
for col in colunas_padronizar:
    df_sem_canc[col + '_standart'] = scaler.fit_transform(df_sem_canc[[col]])

# df_sem_canc

## Coluna taxa de Inadimplência

In [21]:
# Vamos assumir que a taxa de inadimplência seja calculada como a porcentagem da 
# 'ValorParcela' df['Taxa_Inadimplencia'] 
# df_sem_canc.apply(lambda row: row['ValorParcela'] if row['Esteve_Inadimplente?'] else 0, axis=1) 

## One-hot-encoding

In [22]:
# # One-Hot Encoding com matrizes esparsas 
# encoder = OneHotEncoder()

In [23]:
# encoded = encoder.fit_transform(df_sem_canc[['NOME_SEGURADORA_FINAL', 'NmProduto', 'NmTomador']])
# encoded

In [24]:
# # Para converter de volta em DataFrame e juntar com o original 
# df_encoded = pd.DataFrame.sparse.from_spmatrix(encoded, columns=encoder.get_feature_names_out(['NOME_SEGURADORA_FINAL', 'NmProduto', 'NmTomador'])) 
# df_final = pd.concat([df_sem_canc, df_encoded], axis=1)
# df_final

In [25]:
# df_sample = df_sem_canc.sample(frac=0.1, random_state=1)
df_final = pd.get_dummies(df_sem_canc, columns=['NOME_SEGURADORA_FINAL'], drop_first=False)

## Escrita base tratada

In [26]:
df_final.to_csv(r'~\Desktop\tcc_mba_icmc\data\data_curated.csv', index=False)

print("DataFrame escrito com sucesso para 'data_curated.csv'")

DataFrame escrito com sucesso para 'data_curated.csv'


In [27]:
pd.read_csv(r'~\Desktop\tcc_mba_icmc\data\data_curated.csv')

,VlReceitaEsperada,NrParcelas,NmProduto,GrupoProduto,NmTomador,GrupoEconomico,PremioBrutoModDiv,Parcela,ValorParcela,PeriodicidadeMes,...,NOME_SEGURADORA_FINAL_ef29ca0b017f73a647d7b93280e327d67d04c9652505835f839961ffe016abb7,NOME_SEGURADORA_FINAL_ef57e5a22f3c1af190f69cf24780a85ab939296b8a2110f10345c7332612e229,NOME_SEGURADORA_FINAL_efa51d1b726310b3be0cedf18cbabe6c68897d1e2a3cdeff5340db5f5033174b,NOME_SEGURADORA_FINAL_f196974014aad82f9f373bc80500b53092da47b34c67e5a7122ec938fe5775d4,NOME_SEGURADORA_FINAL_f47000b596b290f24cb54e3a8d7f11be387c60d3cfa521817955b6b3c6ae59fa,NOME_SEGURADORA_FINAL_f48e40067a48047401e9e4bcc18fea55cfa0b88e9c3468f7bb0bc897609d712d,NOME_SEGURADORA_FINAL_f6b33afc342d2988e4c750108b804763b86da7fe0a2cd4e46fe4b423874dca2f,NOME_SEGURADORA_FINAL_fb28df294b075e74b80a5c02f4555c6031570106163066c246d482d112cabe1a,NOME_SEGURADORA_FINAL_fe11e9c51a77753fab092e2968abfbe249ba79e8d93ea93273bf3b8bc06ebc94,NOME_SEGURADORA_FINAL_fedb1b2600b69c711cb7b2a5fde86df6c0f8623cd7e525e91915fe485feabc8a
0,0,1,9cdafbed10c0d51c62c70109841e569900f978bb7524ef...,54eb3384177a6f1bce36ebf478c32787a128b43144b901...,dc937b59892604f5a86ac96936cd7ff09e25f18ae6b758...,dc937b59892604f5a86ac96936cd7ff09e25f18ae6b758...,4349,1,0,1,...,False,False,False,False,False,False,False,False,False,False
1,528,1,c68cb6c1bac51b9b18fe707b35d142a54e67f06780c358...,243f77a913aad6b64b4c73b481c94fce10c1554fbe86d1...,7b0c6f6a92137fc18c4accdaf9dc8eb1b7d105496a126e...,e468ad81880f4156af507e2d09f2c6c1746f356460dce0...,2346,1,528,1,...,False,False,False,False,False,False,False,False,False,False
2,66,1,f22e67f434917d011b2a3f8463d63bdf4acf6ddd14840b...,0048e821bb69ffdcd8f340d7d83aaefadea6b47377089b...,418fe50aa1ba9b6660f2afa031b6adf31bf7e7e892d43e...,418fe50aa1ba9b6660f2afa031b6adf31bf7e7e892d43e...,437,1,66,1,...,False,False,False,False,False,False,False,False,False,False
3,818,1,f22e67f434917d011b2a3f8463d63bdf4acf6ddd14840b...,0048e821bb69ffdcd8f340d7d83aaefadea6b47377089b...,9023625c4850288ff84f2dfa62d5bb0a4a6465c82e3c60...,9023625c4850288ff84f2dfa62d5bb0a4a6465c82e3c60...,5453,1,818,1,...,False,False,False,False,False,False,False,False,False,False
4,188,10,0c4302ab1690d261d113a63cb27c17d6e4c04edb378b5b...,541d9e89fcbd48ed2be6049a23f8cffbf4b492adea1c43...,d8629d6d970adfc91f2d7ea3c40ae8004883e9df78555f...,d8629d6d970adfc91f2d7ea3c40ae8004883e9df78555f...,1878,2,19,1,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867274,671,1,d6bd6b18a4384e2bee35d512533d1e90d6d40953b405c8...,541d9e89fcbd48ed2be6049a23f8cffbf4b492adea1c43...,3963ef52df4b1e445148010e5c5ce5206ca795e32ceb1e...,3963ef52df4b1e445148010e5c5ce5206ca795e32ceb1e...,2686,1,671,1,...,False,False,False,False,False,False,False,False,False,False
1867275,852,1,d6bd6b18a4384e2bee35d512533d1e90d6d40953b405c8...,541d9e89fcbd48ed2be6049a23f8cffbf4b492adea1c43...,3963ef52df4b1e445148010e5c5ce5206ca795e32ceb1e...,3963ef52df4b1e445148010e5c5ce5206ca795e32ceb1e...,3410,1,852,1,...,False,False,False,False,False,False,False,False,False,False
1867276,476,1,d6bd6b18a4384e2bee35d512533d1e90d6d40953b405c8...,541d9e89fcbd48ed2be6049a23f8cffbf4b492adea1c43...,3963ef52df4b1e445148010e5c5ce5206ca795e32ceb1e...,3963ef52df4b1e445148010e5c5ce5206ca795e32ceb1e...,1905,1,476,1,...,False,False,False,False,False,False,False,False,False,False
1867277,250,1,d6bd6b18a4384e2bee35d512533d1e90d6d40953b405c8...,541d9e89fcbd48ed2be6049a23f8cffbf4b492adea1c43...,3963ef52df4b1e445148010e5c5ce5206ca795e32ceb1e...,3963ef52df4b1e445148010e5c5ce5206ca795e32ceb1e...,1000,1,250,1,...,False,False,False,False,False,False,False,False,False,False


# Paleta de cores

In [34]:
with open(r"../reports/paleta_cores.json") as f:
    data = json.load(f)
    paleta = data['colors']['dataColors']
    BadPalette = data['colors']['dataColorsBadVariation']
    GoodPalette = data['colors']['dataColorsGoodVariation']
    PrincipalPallete = data['colors']['dataColorsPrincipalVariation']

# Aplicando a paleta personalizada ao Seaborn 
sns.set_palette(paleta)

# Análise

## Visão Geral

In [ ]:
print(
    # df_sem_canc.groupby('Inadimplente_Atual')['ValorParcela'].sum(), 
    '\n', "-"*100, '\n',
    df_sem_canc.groupby('Esteve_Inadimplente?')['ValorParcela'].sum()
    )

In [ ]:
print(
    # df_sem_canc.groupby('Inadimplente_Atual')['PremioBrutoModDiv'].sum(), 
    '\n', "-"*100, '\n',
    df_sem_canc.groupby('Esteve_Inadimplente?')['PremioBrutoModDiv'].sum()
    )

In [ ]:
print(
    # df_sem_canc['Inadimplente_Atual'].value_counts(dropna=False), 
    '\n', "-"*100, '\n',
    df_sem_canc['Esteve_Inadimplente?'].value_counts()
    )

In [ ]:
df['DifData'].apply(lambda x: 'SIM' if x < 0 else 'NÃO').value_counts()



## Matriz de correlação

In [ ]:
df_sem_canc.info()

In [ ]:
df_num = df_sem_canc.select_dtypes(include=['int32', 'int64'])

# Criar a matriz de correlação 
matriz_correlacao = df_num.corr() 
# Plotar a matriz de correlação usando o Seaborn 
plt.figure(figsize=(16, 8)) 
sns.heatmap(matriz_correlacao, annot=True, cmap='coolwarm', center=0) 
plt.title('Matriz de Correlação') 
plt.show()

In [ ]:
# Calcular correlação com a coluna target 
correlacoes = df_num.corr()['Esteve_Inadimplente?_num'].drop('Esteve_Inadimplente?_num') 
# Transformar a série de correlações em um DataFrame para visualização 
matriz_correlacao2 = pd.DataFrame(correlacoes) 
# Renomear a coluna para clareza 
matriz_correlacao2.columns = ['correlation_with_target'] 
plt.figure(figsize=(16, 8)) 
sns.heatmap(matriz_correlacao2, annot=True, cmap='coolwarm', center=0, vmax=matriz_correlacao2.max().iloc[0], vmin=matriz_correlacao2.min().iloc[0])
plt.title('Matriz de Correlação') 
plt.show()

In [ ]:
matriz_correlacao2.sort_values(ascending=False, by='correlation_with_target')

## Análise comissão

In [ ]:
grouped_df = df_final.groupby('Esteve_Inadimplente?')['ValorParcela'].sum().reset_index()
#Criando o gráfico de barras com a paleta personalizada 
fig, ax = plt.subplots(figsize=(8, 6))

# Limite do eixo y
# plt.ylim(0, 110)

# Adicionando uma borda ao redor do gráfico 
rect = patches.FancyBboxPatch(
    (0, 0), 1, 1, 
    boxstyle="round,pad=0.15", edgecolor='#000000', linewidth=1.5, transform=ax.transAxes, clip_on=False,
    facecolor='#ffffff') 
ax.add_patch(rect)
sns.barplot(x='Esteve_Inadimplente?', y='ValorParcela', data=grouped_df, palette=paleta[:2], ax=ax)

# Adicionando os valores nas barras 
for p in ax.patches[-2:]: 
    ax.annotate(
        'R${:,.0f}'.format(p.get_height()), 
        (p.get_x() + p.get_width() / 2., p.get_height()), 
        ha='center', 
        va='baseline', 
        fontsize=10, color='black', xytext=(0, 5), textcoords='offset points')
plt.xticks(rotation=0)
# # Alterando a cor de fundo do gráfico 
ax.set_facecolor('#ffffff')
fig.patch.set_facecolor("#fff8ed")
fig.patch.set_alpha(0.7)

# Adicionando títulos e labels
plt.title('Distribuição Valor Comissão')
plt.xlabel('Inadimplente?')
plt.ylabel('Valor Comissão')

# Exibindo o gráfico
plt.show()


In [ ]:
grouped_df = df_final.groupby('Esteve_Inadimplente?')['ValorParcela'].sum().reset_index()

In [ ]:
df_final[df_final['ValorParcela'] ==  df_final['ValorParcela'].max()]

In [ ]:
for p in ax.patches[-2:]: 
    print('R${:,.0f}'.format(p.get_height()))
    print(f'R${round(p.get_height(), 6)}')

## Análise seguradora

In [ ]:
df_sem_canc.columns

In [ ]:
df_sem_canc[df_sem_canc['Esteve_Inadimplente?'] == 'SIM'].groupby('NOME_SEGURADORA_FINAL')['ValorParcela'].sum().reset_index().sort_values(by='ValorParcela', ascending=False).head()

In [46]:
# Agrupar e somar os valores de 'ValorParcela'
# df_sem_canc.groupby(['NOME_SEGURADORA_FINAL', 'Esteve_Inadimplente?'])['ValorParcela'].sum().reset_index()
seguradoras_maiores_devedoras = df_sem_canc[df_sem_canc['Esteve_Inadimplente?_num'] == 1].groupby(['NOME_SEGURADORA_FINAL', 'Esteve_Inadimplente?'])['ValorParcela'].sum().reset_index().sort_values(by='ValorParcela', ascending=False).head(10)


In [ ]:
df_sem_canc[df_sem_canc['NOME_SEGURADORA_FINAL'].isin(seguradoras_maiores_devedoras['NOME_SEGURADORA_FINAL'][0::])].groupby(['NOME_SEGURADORA_FINAL', 'Esteve_Inadimplente?'])['ValorParcela'].sum().reset_index()

In [ ]:
# Agrupar e somar os valores de 'ValorParcela' para seguradoras maiores devedoras 
df_agrupado = (
    df_sem_canc[df_sem_canc['NOME_SEGURADORA_FINAL'].isin(seguradoras_maiores_devedoras['NOME_SEGURADORA_FINAL'][0::])]
    .groupby(['NOME_SEGURADORA_FINAL', 'Esteve_Inadimplente?'])['ValorParcela']
    .sum()
    .reset_index() 
)

# Dicionário com os valores 
hash_dict = {
    "160a3a39ca881569e42065a74c3fb438090044d57f19d50f98dc5574c4acaab3": 1,
    "228088b037946fe6c4025b8e23f5f01b934561a8c6e441fa950aa8f24513498f": 2,
    "22ac161e69ac52fde9d8db45a533bcab9eebf3f6c5be36e46c86eaf2f7a62dc2": 3,
    "4b8fd8ac52c790f571819a8aecc3784b1dd7d31beda6d2ff47a4939561f928c0": 4,
    "4ef3c8eac2fe13440d323c59d704d51d6f8a5e34ecb55a65c38830d9f110b5bd": 5,
    "58919c823ea092e0a7d32762a06d4bb7a6f369487761f0d30ac16784479a1917": 6,
    "9a704773a99578a4cb7c8d7fd43421921eb6db0967bf6c0d97c49d4432d04078": 7,
    "c9c800b059d082d0d51e221848821313a76bf3d25dc538cc90cb77bdd84b83d4": 8,
    "ce7112b1bb7af451acc6c360998019acfde75d2dcb1527ec32c40605975bcd6e": 9,
    "eece1c4069d46119e73246c56fe3ffadf350638f31276fcf2080cac414743141": 10
}

# Adicionar a coluna 'numero' ao DataFrame df_agrupado 
df_agrupado['numero'] = df_agrupado['NOME_SEGURADORA_FINAL'].map(hash_dict)

# Criar o gráfico de barras usando Seaborn 
plt.figure(figsize=(10, 6)) 
sns.barplot(data=df_agrupado, x='numero', y='ValorParcela', hue='Esteve_Inadimplente?') 
plt.title('Soma de ValorParcela por Seguradora e Status de Inadimplência') 
plt.xlabel('Seguradora') 
plt.ylabel('Soma de ValorParcela') 
plt.legend(title='Esteve Inadimplente?') 
plt.show()

In [ ]:
# Calcular o total de ValorParcela por seguradora
total_por_seguradora = df_sem_canc.groupby('NOME_SEGURADORA_FINAL')['ValorParcela'].sum()

# Calcular o total de ValorParcela onde Esteve_Inadimplente? é 'SIM' por seguradora
total_sim_por_seguradora = df_sem_canc[df_sem_canc['Esteve_Inadimplente?'] == 'SIM'].groupby('NOME_SEGURADORA_FINAL')['ValorParcela'].sum()

# Calcular a taxa de inadimplência por seguradora
taxa_de_inadimplencia = (total_sim_por_seguradora / total_por_seguradora).fillna(0)

# Adicionar a coluna taxa_de_inadimplencia ao DataFrame
df_agrupado = df_agrupado.merge(taxa_de_inadimplencia.rename('taxa_de_inadimplencia'), on='NOME_SEGURADORA_FINAL', how='left', suffixes=('', '_drop')).filter(regex='^(?!.*_drop)')

df_agrupado['taxa_de_inadimplencia'] = df_agrupado['taxa_de_inadimplencia'].round(2)
df_agrupado

In [ ]:
df_agrupado['taxa_de_inadimplencia'].sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(10, 6)) 
ax = sns.barplot(data=df_agrupado, x='numero', y='taxa_de_inadimplencia') 
ax.bar_label(ax.containers[0])
plt.title('Soma de ValorParcela por Seguradora e Status de Inadimplência') 
plt.xlabel('Seguradora') 
plt.ylabel('Soma de ValorParcela') 
plt.legend(title='Esteve Inadimplente?') 
plt.show()



In [ ]:
df_sem_canc['NOME_SEGURADORA_FINAL'].nunique()

## Análise trimestral

In [ ]:
import json
import matplotlib.patheffects as PathEffects
import matplotlib.patches as patches
import seaborn as sns 
import matplotlib.pyplot as plt

with open(r"../reports/paleta_cores.json") as f:
    data = json.load(f)
    paleta = data['colors']['dataColors']
    BadPalette = data['colors']['dataColorsBadVariation']
    GoodPalette = data['colors']['dataColorsGoodVariation']
    PrincipalPallete = data['colors']['dataColorsPrincipalVariation']

def grafico_classes(df, titulo):
    class_counts = df['target'].value_counts(normalize=True) * 100 
    class_counts = class_counts.round(1)
    class_counts = class_counts.reset_index() 
    class_counts.columns = ['target', 'percentual'] 
    class_counts['target'] = class_counts['target'].map({0:'Não', 1:'Sim'})

    # Criando o gráfico de barras com a paleta personalizada 
    fig, ax = plt.subplots(figsize=(8, 6))
    # Adicionando rótulos e título
    plt.xlabel('Inadimplente?')
    plt.ylabel('Quantidade de Inadimplência')
    plt.title(titulo)

    # Limite do eixo y
    plt.ylim(0, 110)

    # Adicionando uma borda ao redor do gráfico 
    rect = patches.FancyBboxPatch(
        (0, 0), 1, 1, 
        boxstyle="round,pad=0.15", edgecolor='#000000', linewidth=1.5, transform=ax.transAxes, clip_on=False,
        facecolor='#ffffff') 
    ax.add_patch(rect)
    sns.barplot(x='target', y='percentual', data=class_counts, palette=paleta[:2], ax=ax)

    # Adicionando os valores nas barras 
    for p in ax.patches[-2:]: 
        ax.annotate(
            f'{p.get_height()}%', 
            (p.get_x() + p.get_width() / 2., p.get_height()), 
            ha='center', 
            va='baseline', 
            fontsize=10, color='black', xytext=(0, 5), textcoords='offset points')

    # # Alterando a cor de fundo do gráfico 
    ax.set_facecolor('#ffffff')
    fig.patch.set_facecolor("#fff8ed")
    fig.patch.set_alpha(0.7)

    # Mostrando o gráfico
    return plt.show()

In [ ]:
df_final[df_final['Esteve_Inadimplente?_num'] == 1].groupby('trimestre').agg({'ValorParcela': 'sum'}).reset_index(names='Trimestre')



In [ ]:


# # Dados de exemplo
# data = df_final[df_final['Esteve_Inadimplente?_num'] == 1].groupby('trimestre').agg({'ValorParcela': 'sum'}).reset_index(names='Trimestre')


# # Calculando os valores cumulativos
# data['Cumulativo'] = np.cumsum(data['ValorParcela'])

# # Adicionando o valor inicial
# data['Cumulativo'] = np.insert(data['Cumulativo'], 0, 0)[:-1]

# # Adicionando a linha de total
# total = data['ValorParcela'].sum()
# # data = data.append({'Trimestre': 'Total', 'ValorParcela': total, 'Cumulativo': data['Cumulativo'].iloc[-1] + total}, ignore_index=True)
# total_row = pd.DataFrame({'Trimestre': ['Total'], 'ValorParcela': [total], 'Cumulativo': [data['Cumulativo'].iloc[-1] + total]})
# df = pd.concat([df, total_row], ignore_index=True)


# # Criando o gráfico de cascata
# fig, ax = plt.subplots(figsize=(10, 6))
# sns.barplot(x='Trimestre', y='ValorParcela', data=df, ax=ax, palette='viridis')

# # Adicionando linhas para conectar as barras
# for i in range(1, len(df['Trimestre'])):
#     ax.plot([i-1, i], [df['Cumulativo'][i-1], df['Cumulativo'][i]], color='black')

# # Ajustando o gráfico
# ax.set_title('Gráfico de Cascata')
# ax.set_xlabel('Trimestre')
# ax.set_ylabel('ValorParcela')

# plt.show()



## Análise Tomador

## Análise Parcelas